In [1]:
from modelzipper.tutils import *

content = auto_read_data("/data/zecheng/data/single_qa/datas.json")

ModelZipper is ready for launch🚀 | Current Version🦄 >>> 0.2.7 <<< | AOE Time🕒 2024-12-22 01:20:49
begin to read data from /data/zecheng/data/single_qa/datas.json | file size: 481.49 MB | file type: json


In [2]:
content.keys()


dict_keys(['qa2', 'qa3', 'qa4', 'qa5', 'qa7'])

In [8]:
content['qa2']['0k']['datas'][0]

{'question': 'The football is in the kitchen.',
 'input': 'Mary journeyed to the bathroom. Sandra went to the garden. Daniel went back to the garden. Daniel went to the office. Sandra grabbed the milk there. Sandra put down the milk there. Daniel went to the hallway. Sandra got the milk there. Daniel went to the garden. Daniel journeyed to the kitchen. Daniel journeyed to the bedroom. Mary journeyed to the garden. Daniel took the football there. Mary moved to the office. Sandra travelled to the bedroom. Daniel dropped the football. Sandra left the milk there. Daniel grabbed the football there. Sandra grabbed the milk there. Daniel went to the kitchen.',
 'golden': 'yes'}